In [1]:
import os
import shutil
import xml.etree.ElementTree as ET
import re

## path init

In [2]:
dataset_path = "/mnt/proj/2017/"
destination_path = "/mnt/proj/llamps/Pre-processing/mind-dataset"
#Source directory
trainset_neg_path = os.path.join(dataset_path,"train/negative_examples_anonymous_chunks")
trainset_pos_path = os.path.join(dataset_path,"train/positive_examples_anonymous_chunks")
testset_path = os.path.join(dataset_path,"test")
testlabel_path = os.path.join(dataset_path,"test/test_golden_truth.txt")
#Destination directory
trainset_neg_des_path = os.path.join(destination_path,"negative_examples_anonymous")
trainset_pos_des_path = os.path.join(destination_path,"positive_examples_anonymous")
testset_des_path = destination_path

## combine trainset

In [3]:
path = trainset_neg_path
#label_text = "2017/train/risk_golden_truth.txt"
savepath = trainset_neg_des_path
if not os.path.exists(savepath):
    os.makedirs(savepath)
flag = "train_subject{user_id}.xml"
flag_ = "train_subject{user_id}_{chunk}.xml"

In [4]:
def intigrate_user(root_path,user):
    user_tree_root = ET.Element("INDIVIDUAL")
    user_id = ET.SubElement(user_tree_root,"ID")
    user_id.text = "train_subject"+str(user)
    for i in range(1,11):
        file_path = os.path.join(root_path,"chunk "+str(i),flag_.format(user_id=user,chunk=i))
        if not os.path.isfile(file_path):
            raise Exception("没这个文件",file_path)
        root = ET.parse(file_path).getroot()
        post_list = root.findall("WRITING")
        for j in range(len(post_list)):
            user_tree_root.append(post_list[len(post_list)-j-1])
    return user_tree_root

In [5]:
def intigrate(root_path,save_path):
    chunk1 = os.path.join(root_path,"chunk 1")
    user_list=[]
    files = os.scandir(chunk1)
    
    for f in files:
        if f.name.split(".")[-1] != "xml":
            print(f.path)
            continue
        a = re.search("(\d{1,4})_",f.name).group(1)
        user_list.append(a)
        
    for user in user_list:
        user_root = intigrate_user(root_path,user)
        user_tree = ET.ElementTree(user_root)
        user_tree.write(os.path.join(save_path,flag.format(user_id=user)))
        

In [6]:
intigrate(path,savepath)

In [7]:
path = trainset_pos_path
savepath = trainset_pos_des_path
if not os.path.exists(savepath):
    os.makedirs(savepath)
intigrate(path,savepath)

## combine testset
Testset is different from trainset. So, code are little different

In [8]:
label_path = testlabel_path
save_path = testset_des_path
root_path = testset_path
if not os.path.exists(save_path):
    os.makedirs(save_path)
flag = "test_subject{user_id}.xml"
flag_ = "test_subject{user_id}_{chunk}.xml"
intigrate(root_path,save_path)

In [9]:
test_neg = "negative_examples_test"
test_pos = "positive_examples_test"
test_neg = os.path.join(save_path,test_neg)
test_pos = os.path.join(save_path,test_pos)
if not os.path.exists(test_neg):
    os.mkdir(test_neg)
if not os.path.exists(test_pos):
    os.mkdir(test_pos)
with open(label_path,'r') as f:
    lines = f.readlines()

In [10]:
for line in lines:
    match = re.search(r"subject(\d{1,4})\t(\d)",line)
    if match:
        #chunk文件夹使用for
        #for i in range(1,11):
        #temp_path = "chunk "+str(i)
        #chunks这里要加chunk
        file_name = "test_subject{id}.xml".format(id=match.group(1))
        #chunks这里要加chunk
        src_path = os.path.join(save_path,file_name)
        if os.path.isfile(src_path):
            if match.group(2)=="1":
                #print(save_path,test_pos,temp_path)
                file_dst = test_pos
                #print(file_dst)
            else:
                file_dst = test_neg
                #print(file_dst)
            if not os.path.exists(file_dst):
                os.mkdir(file_dst)
            file_dst = os.path.join(file_dst,file_name)
        else:
            raise Exception("不是文件",src_path)
        shutil.copy(src_path,file_dst)
        os.remove(src_path)
print("finish")
        
        #print(match.group(1),match.group(2))
   

finish
